In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.api import SimpleExpSmoothing
from prophet import Prophet

c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
tiobe_df = pd.read_csv('new/tiobe.csv')
wiki_df = pd.read_csv('new/wiki.csv')
gtrend_df = pd.read_csv('new/gtrend.csv')
stack_df = pd.read_csv('new/stackoverflow.csv')


In [3]:
languages = list(wiki_df.columns)
languages.remove('Month')
df_list = [tiobe_df, wiki_df, gtrend_df, stack_df]
df_dict = {0:'tiobe', 1:'wikipedia', 2:'gtrend', 3:'stackOverflow'}

di = {}
di['language'] = languages

In [5]:
#ARIMA MODEL

for i, df_all in enumerate(df_list): 
    di['arima_mae_' + df_dict[i]] = []
    di['arima_mse_' + df_dict[i]] = []
    di['arima_rmse_' + df_dict[i]] = []

    for language in languages:
        if language in df_all.columns:
            df = df_all[language]

            splits = 5 if i==3 else 10
            tscv = TimeSeriesSplit(n_splits=splits)
            
            fold_mae, fold_mse, fold_rmse = [],[],[]

            for train_index, test_index in tscv.split(df):
                train, test = df.iloc[train_index], df.iloc[test_index]
                
                model = pm.auto_arima(train, seasonal=False, stepwise=True, trace=False)
                p, d, q = model.order
                
                arima_model = ARIMA(train, order=(p, d, q))
                model = arima_model.fit()

                forecast = model.forecast(steps=len(test))

                mae = mean_absolute_error(test, forecast)
                mse = mean_squared_error(test, forecast)
                rmse = np.sqrt(mse)

                fold_mae.append(mae)
                fold_mse.append(mse)
                fold_rmse.append(rmse)
        
        else:
            fold_mae, fold_mse, fold_rmse = [-1],[-1],[-1]

        avg_mae = np.mean(fold_mae)
        avg_mse = np.mean(fold_mse)
        avg_rmse = np.mean(fold_rmse)

        di['arima_mae_'+ df_dict[i]].append(avg_mae)
        di['arima_mse_' + df_dict[i]].append(avg_mse)
        di['arima_rmse_' + df_dict[i]].append(avg_rmse)

# df_res = pd.DataFrame.from_dict(di)
# df_res

c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as sta

In [6]:
#SIMPLE EXPONENTIAL MODEL

for i, df_all in enumerate(df_list): 
    di['ses_mae_' + df_dict[i]] = []
    di['ses_mse_' + df_dict[i]] = []
    di['ses_rmse_' + df_dict[i]] = []

    for language in languages:
        if language in df_all.columns:
            df1 = df_all[language]
            mae_list, mse_list, rmse_list = [], [], []

            splits = 5 if i==3 else 10
            tscv = TimeSeriesSplit(n_splits=splits)
            
            for train_index, test_index in tscv.split(df1):
                train, test = df1[train_index], df1[test_index]
                
                ses = SimpleExpSmoothing(train)
                model = ses.fit(optimized=True)
                
                forecast = model.forecast(steps=len(test))
                
                mae = mean_absolute_error(test, forecast)
                mse = mean_squared_error(test, forecast)
                rmse = np.sqrt(mse)
                
                mae_list.append(mae)
                mse_list.append(mse)
                rmse_list.append(rmse)
        
        else:
            mae_list, mse_list, rmse_list = [-1],[-1],[-1]

        avg_mae = np.mean(mae_list)
        avg_mse = np.mean(mse_list)
        avg_rmse = np.mean(rmse_list)
        
        di['ses_mae_'+ df_dict[i]].append(avg_mae)
        di['ses_mse_' + df_dict[i]].append(avg_mse)
        di['ses_rmse_' + df_dict[i]].append(avg_rmse)


c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\

In [7]:
# HOLT-WINTERS MODEL

for i, df_all in enumerate(df_list): 
    di['holt_mae_' + df_dict[i]] = []
    di['holt_mse_' + df_dict[i]] = []
    di['holt_rmse_' + df_dict[i]] = []

    for language in languages:
        if language in df_all.columns:
            df1 = df_all[language]
            mae_list, mse_list, rmse_list = [], [], []

            splits = 5 if i==3 else 10
            tscv = TimeSeriesSplit(n_splits=splits)
            
            for train_index, test_index in tscv.split(df1):
                train, test = df1[train_index], df1[test_index]
                
                holt_model = ExponentialSmoothing(train, trend="additive", seasonal=None)
                model = holt_model.fit(optimized=True)
                
                forecast = model.forecast(steps=len(test))
                
                mae = mean_absolute_error(test, forecast)
                mse = mean_squared_error(test, forecast)
                rmse = np.sqrt(mse)
                
                mae_list.append(mae)
                mse_list.append(mse)
                rmse_list.append(rmse)
        
        else:
            mae_list, mse_list, rmse_list = [-1],[-1],[-1]

        avg_mae = np.mean(mae_list)
        avg_mse = np.mean(mse_list)
        avg_rmse = np.mean(rmse_list)
        
        di['holt_mae_'+ df_dict[i]].append(avg_mae)
        di['holt_mse_' + df_dict[i]].append(avg_mse)
        di['holt_rmse_' + df_dict[i]].append(avg_rmse)


c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\holtwinters\

In [8]:
# PROPHET MODEL

for i, df_all in enumerate(df_list): 
    di['prophet_mae_' + df_dict[i]] = []
    di['prophet_mse_' + df_dict[i]] = []
    di['prophet_rmse_' + df_dict[i]] = []

    for language in languages:
        if language in df_all.columns:
            tm = 'year' if i==3 else 'Month'
            df1 = df_all[[tm, language]]
            mae_list, mse_list, rmse_list = [], [], []

            splits = 5 if i==3 else 10
            tscv = TimeSeriesSplit(n_splits=splits)
            
            for train_index, test_index in tscv.split(df1):
                train, test = df1.iloc[train_index], df1.iloc[test_index]

                prophet_df = train.reset_index(drop=True)
                prophet_df.columns = ['ds', 'y']

                prophet_model = Prophet()
                prophet_model.fit(prophet_df)

                future = prophet_model.make_future_dataframe(periods=len(test))
                forecast = prophet_model.predict(future)

                test_df = test.reset_index(drop=True)
                test_df.columns = ['ds', 'y']

                forecast_test = forecast[-len(test):]

                mae = mean_absolute_error(test_df['y'], forecast_test['yhat'])
                mse = mean_squared_error(test_df['y'], forecast_test['yhat'])
                rmse = np.sqrt(mse)

                mae_list.append(mae)
                mse_list.append(mse)
                rmse_list.append(rmse)
        
        else:
            mae_list, mse_list, rmse_list = [-1],[-1],[-1]

        avg_mae = np.mean(mae_list)
        avg_mse = np.mean(mse_list)
        avg_rmse = np.mean(rmse_list)
        
        di['prophet_mae_'+ df_dict[i]].append(avg_mae)
        di['prophet_mse_' + df_dict[i]].append(avg_mse)
        di['prophet_rmse_' + df_dict[i]].append(avg_rmse)




00:10:20 - cmdstanpy - INFO - Chain [1] start processing
00:10:21 - cmdstanpy - INFO - Chain [1] done processing
00:10:21 - cmdstanpy - INFO - Chain [1] start processing
00:10:21 - cmdstanpy - INFO - Chain [1] done processing
00:10:21 - cmdstanpy - INFO - Chain [1] start processing
00:10:21 - cmdstanpy - INFO - Chain [1] done processing
00:10:21 - cmdstanpy - INFO - Chain [1] start processing
00:10:21 - cmdstanpy - INFO - Chain [1] done processing
00:10:22 - cmdstanpy - INFO - Chain [1] start processing
00:10:22 - cmdstanpy - INFO - Chain [1] done processing
00:10:22 - cmdstanpy - INFO - Chain [1] start processing
00:10:22 - cmdstanpy - INFO - Chain [1] done processing
00:10:22 - cmdstanpy - INFO - Chain [1] start processing
00:10:22 - cmdstanpy - INFO - Chain [1] done processing
00:10:22 - cmdstanpy - INFO - Chain [1] start processing
00:10:22 - cmdstanpy - INFO - Chain [1] done processing
00:10:22 - cmdstanpy - INFO - Chain [1] start processing
00:10:22 - cmdstanpy - INFO - Chain [1]

In [9]:
df_res = pd.DataFrame.from_dict(di)
df_res.to_csv('result_timeseries.csv', index=False)
df_res.head()

,language,arima_mae_tiobe,arima_mse_tiobe,arima_rmse_tiobe,arima_mae_wikipedia,arima_mse_wikipedia,arima_rmse_wikipedia,arima_mae_gtrend,arima_mse_gtrend,arima_rmse_gtrend,...,prophet_rmse_tiobe,prophet_mae_wikipedia,prophet_mse_wikipedia,prophet_rmse_wikipedia,prophet_mae_gtrend,prophet_mse_gtrend,prophet_rmse_gtrend,prophet_mae_stackOverflow,prophet_mse_stackOverflow,prophet_rmse_stackOverflow
0,Java,0.119842,0.025445,0.136695,0.178869,0.049087,0.212511,0.061121,0.008737,0.068036,...,0.240213,0.427435,0.357488,0.490990,0.235784,0.107352,0.259473,7.058014,54.809452,7.058014
1,JavaScript,0.210500,0.067358,0.243572,0.102025,0.034808,0.119430,0.125514,0.036125,0.156291,...,0.480923,0.220635,0.106856,0.237779,0.349064,0.249646,0.387360,6.647655,45.436382,6.647655
2,PHP,0.102340,0.027085,0.115292,0.105663,0.019045,0.128879,0.084962,0.012336,0.098950,...,0.379585,0.224073,0.093525,0.254016,0.298533,0.182748,0.328788,6.945215,51.405985,6.945215
3,Python,0.066918,0.008777,0.079447,0.115188,0.038033,0.135180,0.103449,0.023095,0.128905,...,0.135901,0.189092,0.120477,0.203952,0.216651,0.098039,0.243536,6.859527,47.749094,6.859527
4,MySQL,-1.000000,-1.000000,-1.000000,0.105834,0.020038,0.128611,0.117110,0.022642,0.140118,...,-1.000000,0.382661,0.279948,0.428681,0.460319,0.476246,0.511377,7.541543,59.558630,7.541543
